#### PROJETO ETL - PYTHON ( Extração - Transformação - Carregamento de dados)
*Etapas Extração - Validação - Limpeza -Transformação - carregamento de Dados*

1 - <b>Extract:</b> Os dados são extraidos de diferentes fontes de dados.

2 - <b>Transform:</b> Propagados para área de preparação de dados, onde são transformados e limpos.

3 - <b>Load:</b> Carregados no data Warehouse - Cluster Hadoop

##### Ferramentas ETL: Apache Airflow, pacote luigi, bonobo, bubbles, petl, pandas 

### OCORRÊNCIA AERONAUTICA NA AVIAÇÃO BRASILEIRA CENIPA

* FONTE DADOS: http://sistema.cenipa.aer.mil.br/cenipa/media/opendata/ocorrencia.csv


### 1º Extração dos dados

In [1]:
import pandas as pd
import pandera as pa
#pandas por padrão o separador é (,) então precisamos passar parametro (sep=;) pois a planilha está com ;
# parse_dates = converter o campo em data
#dayfirst = colocar a data em primeiro pois no pandas é em ultimo padrao americano
df = pd.read_csv("ocorrencia.csv",nrows=2000, sep=";", parse_dates=['ocorrencia_dia'], dayfirst=True)

In [8]:
# primeiras 10 linhas .head ultimas .tail
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,...,2010-01-03,12:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
5,39807,39807,39807,39807,39807,INCIDENTE,***,***,SALVADOR,BA,...,2010-01-06,17:53:00,***,NaN,NaN,NÃO,NaN,0,1,NÃO
6,40215,40215,40215,40215,40215,INCIDENTE,***,***,COARI,AM,...,2010-01-07,18:40:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,SIM
7,39707,39707,39707,39707,39707,INCIDENTE GRAVE,-6.5319444444,-64.3805555556,CANUTAMA,AM,...,2010-01-09,12:30:00,SIM,FINALIZADA,IG-028/CENIPA/2011,SIM,2011-07-21,3,1,NÃO
8,39156,39156,39156,39156,39156,INCIDENTE GRAVE,-15.2402777778,-59.3541666667,CASCAVEL,PR,...,2010-01-10,23:15:00,SIM,FINALIZADA,I-004/CENIPA/2011,SIM,2011-06-30,2,1,NÃO
9,39711,39711,39711,39711,39711,INCIDENTE GRAVE,***,***,PARÁ DE MINAS,MG,...,2010-01-10,20:00:00,***,NaN,NaN,NÃO,NaN,0,1,NÃO


In [9]:
#criando o schema para data
#premitir itens em branco nullable=True
schema = pa.DataFrameSchema(
columns={
    "codigo": pa.Column(pa.Int, required=False),
    "codigo_ocorrencia": pa.Column(pa.Int),
    "codigo_ocorrencia2":pa.Column(pa.Int),
    "ocorrencia_classificacao": pa.Column(pa.String),
    "ocorrencia_cidade":pa.Column(pa.String),
    "ocorrencia_cidade":pa.Column(pa.String),
    "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2)),
    "ocorrencia_dia":pa.Column(pa.DateTime),
    "ocorrencia_hora":pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'),nullable=True),
    "total_recomendacoes":pa.Column(pa.Int)
    }
)

In [10]:
#validando o dataFrmae
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia1,codigo_ocorrencia2,codigo_ocorrencia3,codigo_ocorrencia4,ocorrencia_classificacao,ocorrencia_latitude,ocorrencia_longitude,ocorrencia_cidade,ocorrencia_uf,...,ocorrencia_dia,ocorrencia_hora,investigacao_aeronave_liberada,investigacao_status,divulgacao_relatorio_numero,divulgacao_relatorio_publicado,divulgacao_dia_publicacao,total_recomendacoes,total_aeronaves_envolvidas,ocorrencia_saida_pista
0,40211,40211,40211,40211,40211,INCIDENTE,***,***,RIO DE JANEIRO,RJ,...,2010-01-03,12:00:00,SIM,FINALIZADA,***,NÃO,NaN,0,1,NÃO
1,40349,40349,40349,40349,40349,INCIDENTE,NaN,NaN,BELÉM,PA,...,2010-01-03,11:05:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
2,40351,40351,40351,40351,40351,INCIDENTE,NaN,NaN,RIO DE JANEIRO,RJ,...,2010-01-03,03:00:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
3,39527,39527,39527,39527,39527,ACIDENTE,-13.1066666667,-55.9930555556,LUCAS DO RIO VERDE,MT,...,2010-01-04,17:30:00,SIM,FINALIZADA,A-539/CENIPA/2018,SIM,2019-10-28,0,1,NÃO
4,40324,40324,40324,40324,40324,INCIDENTE,NaN,NaN,PELOTAS,RS,...,2010-01-05,19:25:00,SIM,FINALIZADA,NaN,NÃO,NaN,0,1,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,47659,47659,47659,47659,47659,ACIDENTE,-23.506667°,-46.633889°,CAJAMAR,SP,...,2013-04-22,14:00:00,***,ATIVA,A-077/CENIPA/2013,NÃO,NaN,0,1,NÃO
1996,47661,47661,47661,47661,47661,ACIDENTE,-6.6294444444,-72.7055555556,ATALAIA DO NORTE,AM,...,2013-04-22,19:00:00,SIM,FINALIZADA,A-035/CENIPA/2013,SIM,2013-08-14,2,1,NÃO
1997,47699,47699,47699,47699,47699,INCIDENTE GRAVE,-1.3847222222,-48.4788888889,BELÉM,PA,...,2013-04-22,11:28:00,***,FINALIZADA,IG-076/CENIPA/2013,SIM,2013-09-16,2,1,SIM
1998,47979,47979,47979,47979,47979,ACIDENTE,***,***,ITAPIRAPUÃ,GO,...,2013-04-22,20:30:00,***,NaN,NaN,NÃO,NaN,0,1,NÃO


In [ ]:
#verificar tipos de dados que foram extraidos (dtypes)

df.dtypes

In [ ]:
#arrumar a data = usar read_csv parametro parse_dates=[] (pegando o mes da data dt.month) parse só funciona se a data estiver em padrao correto
df.ocorrencia_dia.dt.month

### 2º Transformação dos dados (validação dos dados)

In [ ]:
#parametro dayfirst=True para arrumar a data pois na planilha a data é primeiro e no pandas o ultimo
#texto vazio vem NaN data vazio vem NaT
# importar lib pandera para criar schema de validação de data.
#validar itens em branco nullable=True
#validar hora com pa.Check.str_matches(expressão regular)
#validar tamnha dado ex: Uf 2 caracteres função .Check() função de restição str_length(2,2)
# validar coluna nem sempre vem ( parametro required=False) padrão coluna required true


### 3º Limpeza de dados

*remover * # em branco siglas incorretas etc.. *colocar um nome diferente*
depende de cada projeto



4